In [14]:
import pandas as pd
import numpy as np
import gc
from sklearn import preprocessing
from IPython.display import clear_output
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.linear_model import LinearRegression
from utils.age_utils import transform_age, anti_transform_age
from utils.metrics import evaluate_age_prediction
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Data preperation

In [2]:
geo_data_27k = [
    'GSE27317',
    'GSE41037',
    'GSE38873',
    'GSE15745',
    'GSE32393',
    'GSE25892',
    'GSE20242',
    'GSE22595',
    'GSE37988',
    'GSE17448',
    'GSE36642',
    'GSE26126',
    'GSE34035',
    'GSE28746',
    'GSE20236',
    'GSE19711',
    'GSE37008',
    'GSE36812',
    'GSE34257',
    'GSE38608',
    'GSE38291',
    'GSE36166',
    'GSE63384',
    'GSE59274',
    'GSE57285',
    'GSE56606',
    'GSE49908',
    'GSE49907',
    'GSE49905',
    'GSE49904'
]

geo_data_450k = [
    'GSE90124',
    'GSE115797',
    'GSE99624',
    'GSE108213',
    'GSE92767',
    'GSE69176',
    'GSE40360',
    'GSE59157',
    'GSE42861',
    'GSE77241',
    'GSE148000'
]

array_express_27k = [
    'E-GEOD-43256',
    'E-GEOD-64940',
    'E-MTAB-2344',
    'E-GEOD-62867',
    'E-GEOD-44763',
    'E-GEOD-48988',
    'E-GEOD-58119',
    'E-GEOD-58045',
    'E-GEOD-57484',
    'E-GEOD-54211',
    'E-GEOD-56342',
    'E-GEOD-27044',
    'E-GEOD-36194',
    'E-GEOD-21232',
    'E-GEOD-32867',
    'E-GEOD-30759',
    'E-GEOD-30758',
    'E-GEOD-32396',
    'E-GEOD-31979',
    'E-MTAB-625',
    'E-MTAB-487'
]

array_express_450k = [
    'E-MTAB-2372',
    'E-GEOD-73832',
    'E-GEOD-71678',
    'E-GEOD-71245',
    'E-GEOD-83334',
    'E-GEOD-75248',
    'E-GEOD-77955',
    'E-GEOD-67705',
    'E-GEOD-77445',
    'E-GEOD-79056',
    'E-GEOD-72556',
    'E-GEOD-52068',
    'E-GEOD-74738',
    'E-GEOD-76105',
    'E-GEOD-65638',
    'E-GEOD-71955',
    'E-GEOD-63106',
    'E-GEOD-73377',
    'E-GEOD-56515',
    'E-GEOD-73103',
    'E-GEOD-67024',
    'E-GEOD-72338',
    'E-GEOD-59457',
    'E-GEOD-64511',
    'E-GEOD-64495',
    'E-GEOD-59509',
    'E-GEOD-67444',
    'E-GEOD-62219',
    'E-GEOD-51954',
    'E-GEOD-52588',
    'E-GEOD-36054',
    'E-GEOD-50660',
    'E-GEOD-61259',
    'E-GEOD-61258',
    'E-GEOD-61257',
    'E-GEOD-61454',
    'E-GEOD-61380',
    'E-GEOD-61107',
    'E-GEOD-54690',
    'E-GEOD-49149',
    'E-GEOD-55438',
    'E-GEOD-53740',
    'E-GEOD-57767',
    'E-GEOD-49064',
    'E-GEOD-50759',
    'E-GEOD-56553',
    'E-GEOD-54399',
    'E-GEOD-53162',
    'E-GEOD-53128',
    'E-GEOD-50498',
    'E-GEOD-47513',
    'E-GEOD-49393',
    'E-GEOD-39004',
    'E-GEOD-51388',
    'E-GEOD-51032',
    'E-GEOD-48325',
    'E-GEOD-44712',
    'E-GEOD-45461',
    'E-GEOD-40279',
    'E-GEOD-41169',
    'E-GEOD-32149',
    'E-GEOD-41826',
    'E-GEOD-42700',
    'E-GEOD-32146',
    'E-GEOD-30870',
    'E-GEOD-34639',
    'E-GEOD-63347',
    'E-GEOD-59592'
]

tcga_all = [
    'TGCA_LUSC',
    'TGCA_THCA',
    'TGCA_HNSC',
    'TGCA_KIRC',
    'TGCA_KIRP',
    'TGCA_LUAD',
    'TGCA_PRAD',
    'TGCA_STAD',
    'TGCA_COAD',
    'TGCA_LIHC',
    'TGCA_UCEC',
    'TGCA_BRCA'
]

cancer_data = [
    'GSE32393',
    'GSE37988',
    'GSE26126',
    'GSE63384',
    'GSE59157',
    'E-GEOD-32867',
    'E-GEOD-30759',
    'E-GEOD-31979',
    'E-GEOD-77955',
    'E-GEOD-52068',
    'E-GEOD-49149',
    'E-GEOD-39004'
]

cancer_comparison = [
    'GSE53051',
]

reliability = [
    'GSE55763',
]

rejuvenation = [
    'GSE142439',
    'GSE116754',
    'GSE65214',
    'GSE44430',
    'GSE45727',
    'GSE30653',
    'GSE37066',
    'GSE30456',
]

senescence_analysis = [
    'GSE91069',
    'GSE100249'
]

data_27k = np.concatenate([np.array(geo_data_27k), np.array(array_express_27k)])
data_450k = np.concatenate([np.array(geo_data_450k), np.array(array_express_450k)])
geo_data = np.concatenate([data_27k, data_450k])
tcga_all = np.array(tcga_all)
all_data = np.concatenate([data_27k, data_450k, tcga_all])

In [6]:
#load all train data for model training
count = 0
for dataset in all_data:
    print(dataset)
    if count == 0:
        all_train = pd.read_pickle('../data_train/' + dataset + '.pkl')
        count += 1
    else:
        new_df = pd.read_pickle('../data_train/' + dataset + '.pkl')
        all_train = pd.concat([all_train, new_df], join ='inner')
        gc.collect()
    clear_output()

#load all test data
count = 0
for dataset in all_data:
    print(dataset)
    if count == 0:
        all_test = pd.read_pickle('../data_test/' + dataset + '.pkl')
        count += 1
    else:
        new_df = pd.read_pickle('../data_test/' + dataset + '.pkl')
        all_test = pd.concat([all_test, new_df], join ='inner')
        gc.collect()
    clear_output()

In [7]:
multi_platform_cpgs = np.array(pd.read_pickle('../dependencies/multi_platform_cpgs.pkl'))

#extracting age and removing unimportant columns
train_ages = all_train['age'].astype('float64')
test_ages = all_test['age'].astype('float64')

train_info = all_train[['dataset', 'tissue_type', 'age', 'gender']]
test_info = all_test[['dataset', 'tissue_type', 'age', 'gender']]

train = all_train[multi_platform_cpgs]
test = all_test[multi_platform_cpgs]

In [8]:
train_cols = train.columns
train_index = train.index

test_cols = test.columns
test_index = test.index

In [9]:
#scaling the data so each columns has 0 mean and variance 1
scaler = preprocessing.RobustScaler()

train_scaled = pd.DataFrame(scaler.fit_transform(train), index = train_index, columns = train_cols)
test_scaled = pd.DataFrame(scaler.transform(test), index = test_index, columns = test_cols)

## Prepare Horvath's CpG data

In [10]:
# Load the CSV file
coeff_df = pd.read_csv("../dependencies/horvath_cpg.csv", skiprows=2)  # skip rows for metadata above the header

# Get the list of CpG IDs (excluding the intercept row)
horvath_cpg_list = coeff_df['CpGmarker'].dropna().tolist()
horvath_cpg_list = [cpg for cpg in horvath_cpg_list if cpg != '(Intercept)']

In [11]:
train_horvath = all_train[horvath_cpg_list]
test_horvath = all_test[horvath_cpg_list]

train_cols_horvath = train_horvath.columns
train_index_horvath = train_horvath.index

test_cols_horvath = test_horvath.columns
test_index_horvath = test_horvath.index

In [12]:
scaler_horvath = preprocessing.RobustScaler()

train_horvath_scaled = pd.DataFrame(scaler_horvath.fit_transform(train_horvath), index = train_index_horvath, columns = train_cols_horvath)
test_horvath_scaled = pd.DataFrame(scaler_horvath.transform(test_horvath), index = test_index_horvath, columns = test_cols_horvath)

# Models

# Decision Trees

In [15]:
tree_model = DecisionTreeRegressor(max_depth=5, random_state=42)
tree_model.fit(train_horvath, train_ages)

DecisionTreeRegressor(max_depth=5, random_state=42)

In [16]:
result = anti_transform_age(tree_model.predict(test_horvath))

res = evaluate_age_prediction(test_ages, result, model_name="decision tree", cpg_count=len(train_cols_horvath))
print(res.round(3))

           Model  CpGs      MAD    MAE         MSE  Pearson R  Median Error
0  decision tree   353  997.809  841.8  939026.858      0.878      -997.809


## Random Forest

In [17]:
forest_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
forest_model.fit(train_horvath, train_ages)

RandomForestRegressor(max_depth=5, random_state=42)

In [18]:
result_forest = anti_transform_age(forest_model.predict(test_horvath))

res_forest = evaluate_age_prediction(test_ages, result_forest, model_name="decision tree", cpg_count=len(train_cols_horvath))
print(res_forest.round(3))

           Model  CpGs       MAD      MAE        MSE  Pearson R  Median Error
0  decision tree   353  1001.604  843.273  924422.16      0.912     -1001.604


## XGBoost

## XGBoost, horvath's CpG

In [52]:
import xgboost as xgb

dtrain = xgb.DMatrix(data=train_horvath, label=transform_age(train_ages))

params = {
    'tree_method': 'hist',
    'device' : 'cuda',
    'predictor':'gpu_predictor',
    'max_depth': 4,
    'learning_rate': 0.1,
}

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100
)


C:\Users\elena\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:26:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [53]:
dtest = xgb.DMatrix(test_horvath)

result = anti_transform_age(model.predict(dtest))

res = evaluate_age_prediction(test_ages, result, model_name="XGBoost", cpg_count=len(train_cols_horvath))
print(res.round(3))

     Model  CpGs    MAD   MAE     MSE  Pearson R  Median Error
0  XGBoost   353  3.443  5.09  57.252      0.961         0.022


In [ ]:
from sklearn.model_selection import GridSearchCV
model = xgb.XGBRegressor(
    tree_method='hist',
    device = 'cuda',
    predictor='gpu_predictor'
)

param_grid = {
    'max_depth': [7, 8, 9],
    'learning_rate': [0.1, 0.2],
    'n_estimators': [100]
}

grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(train_horvath, transform_age(train_ages))

print(grid_search.best_params_)

model.fit(train_horvath, transform_age(train_ages))

In [55]:
result = anti_transform_age(model.predict(test_horvath))

res = evaluate_age_prediction(test_ages, result, model_name="XGBoost", cpg_count=len(train_cols_horvath))
print(res.round(3))

     Model  CpGs    MAD    MAE     MSE  Pearson R  Median Error
0  XGBoost   353  3.426  5.111  58.517       0.96         0.006


In [40]:
model = xgb.XGBRegressor(
    tree_method='hist',
    device = 'cuda',
    predictor='gpu_predictor',
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1
)

model.fit(train_horvath, transform_age(train_ages))

C:\Users\elena\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:05:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [33]:
result = anti_transform_age(model.predict(test_horvath))

res = evaluate_age_prediction(test_ages, result, model_name="XGBoost", cpg_count=len(train_cols_horvath))
print(res.round(3))

     Model  CpGs    MAD    MAE     MSE  Pearson R  Median Error
0  XGBoost   353  3.524  5.054  56.231      0.962         0.026


## XGBoost, all CpG's

In [38]:
import xgboost as xgb

dtrain = xgb.DMatrix(data=train, label=transform_age(train_ages))

params = {
    'tree_method': 'hist',
    'device' : 'cuda',
    'predictor':'gpu_predictor',
    'max_depth': 4,
    'learning_rate': 0.1,
}

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=100
)


C:\Users\elena\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:02:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [39]:
dtest = xgb.DMatrix(test)

result = anti_transform_age(model.predict(dtest))

res = evaluate_age_prediction(test_ages, result, model_name="XGBoost", cpg_count=len(train_cols_horvath))
print(res.round(3))

     Model  CpGs    MAD    MAE     MSE  Pearson R  Median Error
0  XGBoost   353  3.414  5.004  55.422      0.962         0.019


In [42]:
model = xgb.XGBRegressor(
    tree_method='hist',
    device = 'cuda',
    predictor='gpu_predictor',
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1
)

model.fit(train, transform_age(train_ages))

C:\Users\elena\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:13:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [43]:
result = anti_transform_age(model.predict(test))

res = evaluate_age_prediction(test_ages, result, model_name="XGBoost", cpg_count=len(train_cols_horvath))
print(res.round(3))

     Model  CpGs    MAD    MAE    MSE  Pearson R  Median Error
0  XGBoost   353  3.446  4.994  54.95      0.963         0.024
